In [1]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
import os
import numpy as np
import pandas as pd
import csv
import itertools
import sys
from pandas import DataFrame
from keras.models import Sequential

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
%matplotlib inline
#from imutils import paths
import argparse
import random
import pickle
import cv2

from bs4 import BeautifulSoup 
import re # For regular expressions


# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer


#For Tokenizing and pad sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



from socket import *
import socket
from _thread import *
import threading 
import json
from io import StringIO



import string

#For Lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

Using TensorFlow backend.


In [4]:
import pandas as pd
text_emotion = pd.read_csv("text_emotion.csv")
text_emotion=text_emotion[text_emotion.sentiment != 'enthusiasm']
text_emotion=text_emotion[text_emotion.sentiment != 'neutral']
text_emotion=text_emotion[text_emotion.sentiment != 'empty']
text_emotion=text_emotion[text_emotion.sentiment != 'surprise']
text_emotion=text_emotion[text_emotion.sentiment != 'boredom']
text_emotion=text_emotion[text_emotion.sentiment != 'relief']
text_emotion=text_emotion[text_emotion.sentiment != 'worry']
text_emotion=text_emotion[text_emotion.sentiment != 'love']
text_emotion=text_emotion[text_emotion.sentiment != 'fun']
text_emotion['sentiment'] = text_emotion['sentiment'].map({'sadness': "sadness",'hate':'anger',
                                                           'happiness': "joy",
                                                           'anger': "anger"})
text_emotion=text_emotion.drop(columns=['tweet_id', 'author'])
text_emotion = text_emotion.reindex(sorted(text_emotion.columns), axis=1)
text_emotion.columns = ['content', 'Affect']

In [5]:
import pandas as pd
test = pd.read_csv("DS-Test.csv",encoding='ISO-8859-1')
train = pd.read_csv("DS-Training.csv",encoding='ISO-8859-1')
train = train[train.Intensity != 0]
train=train.append({'ID' : '7102' , 'content' : 'im happy','Affect' : 'joy','Intensity' : 0},ignore_index=True)
train=train.drop(columns=['ID', 'Intensity'])
train=train.append(text_emotion)
train1=pd.read_csv("DS-Training.csv",encoding='ISO-8859-1')
train1 = train1[train1.Intensity != 0]
train1=train1.append({'ID' : '7102' , 'content' : 'im happy','Affect' : 'joy','Intensity' : 0},ignore_index=True)

train1

,ID,content,Affect,Intensity
0,2017-En-40427,@VivYau is it all doom and gloom? I only want ...,sadness,1
1,2017-En-21180,The 2nd step to beating #anxiety or #depressio...,sadness,1
2,2017-En-10175,Forgot to eat dinner and now I'm furious with ...,anger,3
3,2017-En-31406,Your future is bright. #Remember,joy,2
4,2017-En-30487,Check out this #film Robocoq 301 #animated #sh...,joy,1
5,2017-En-40386,@jccrocker #CharlotteProtest do u #wait 4 the ...,sadness,1
6,2017-En-11234,@russbully Ended up paying 75p for half a tube...,anger,2
7,2017-En-21999,Tho we haven't talked Jeff but the news is so ...,fear,3
8,2017-En-10288,luv seeing a man with a scowl on his face walk...,anger,2
9,2017-En-20033,Just want Saturday to be over but then again I...,fear,3


In [6]:
# This function converts a text to a sequence of words.
def review_wordlist(review, remove_stopwords=False):
    # Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # Converting to lower case and splitting
    trains_content = review_text.lower().split()
    # Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        trains_content = [w for w in trains_content if not w in stops]

    
    return(trains_content)


# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data
#nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    # This returns the list of lists
    return sentences



In [7]:
#Removing punctuation problems
train["content"] = train["content"].str.replace('[^\w\s]','')
train["content"].head()

#Removing common words
freq = pd.Series(' '.join(train["content"]).split()).value_counts()[:10]
freq = list(freq.index)
train["content"] = train["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train["content"].head()

#Removing rarely occuring words
freq = pd.Series(' '.join(train["content"]).split()).value_counts()[-10:]
freq = list(freq.index)
train["content"] = train["content"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
train["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
train["content"] = train["content"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
print (train["content"])




sentences = []
print("Parsing sentences from training set")
for review in train["content"]:
    sentences += review_sentences(review, tokenizer)
    

0        VivYau it all doom gloom only want hear lovely...
1        The 2nd step beating anxiety or depression rea...
2        Forgot eat dinner now Im furious with everythi...
3                              Your future bright Remember
4        Check out this film Robocoq 301 animated short...
5        jccrocker CharlotteProtest do u wait 4 fact vi...
6        russbully Ended up paying 75p for half tube sm...
7        Tho we havent talked Jeff but news so sad shoc...
8        luv seeing man with scowl on his face walking ...
9        Just want Saturday be over but then again want...
10       love mother but talking about recent horrific ...
11       Will WHU be old bill free by time game with Ch...
12       Today reached 1000 subscriber on YT goodday th...
13       On bedrest since got out hospital U find unope...
14                         Idubbbz LeafyIsHere am offended
15       MissFushiGaming hashtag thing kid always tell ...
16               Do what make successful happy now forev

In [8]:
#for intensity
#Removing punctuation problems
train1["content"] = train1["content"].str.replace('[^\w\s]','')
train1["content"].head()

#Removing common words
freq1 = pd.Series(' '.join(train1["content"]).split()).value_counts()[:10]
freq1 = list(freq1.index)
train1["content"] = train1["content"].apply(lambda x1: " ".join(x1 for x1 in x1.split() if x1 not in freq1))
train1["content"].head()

#Removing rarely occuring words
freq1 = pd.Series(' '.join(train1["content"]).split()).value_counts()[-10:]
freq1 = list(freq1.index)
train1["content"] = train1["content"].apply(lambda x1: " ".join(x1 for x1 in x1.split() if x1 not in freq))
train1["content"].head()

#Lemmatizaton is the much better option, it add the words into the root words
train1["content"] = train1["content"].apply(lambda x1: " ".join([Word(word).lemmatize() for word in x1.split()]))
print (train1["content"])




sentences1 = []
print("Parsing sentences from training set")
for review in train1["content"]:
    sentences1 += review_sentences(review, tokenizer)

0       VivYau it all doom gloom only want hear lovely...
1       The 2nd step beating anxiety or depression rea...
2       Forgot eat dinner now Im furious with everythi...
3                             Your future bright Remember
4       Check out this film Robocoq 301 animated short...
5       jccrocker CharlotteProtest do u wait 4 fact vi...
6       russbully Ended up paying 75p for half tube sm...
7       Tho we havent talked Jeff but news so sad shoc...
8       luv seeing man with scowl on his face walking ...
9       Just want Saturday be over but then again want...
10      love mother but talking about recent horrific ...
11      Will WHU be old bill free by time game with Ch...
12      Today reached 1000 subscriber on YT goodday th...
13      On bedrest since got out hospital U find unope...
14                        Idubbbz LeafyIsHere am offended
15      MissFushiGaming hashtag thing kid always tell ...
16              Do what make successful happy now forever
17      Hey al

In [9]:
# Importing the built-in logging module for saving the state of the model
import logging
import gensim.downloader as api
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [10]:
# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

Training model....


In [11]:
#for intensity
# Creating the model and setting values for the various parameters
num_features1 = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model1 = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model1.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name1 = "300features_40minwords_10context"
model1.save(model_name1)

Training model....


In [12]:
model1.wv.syn0.shape

C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(636, 300)

In [13]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [14]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in train['content']:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)
#print (trainDataVecs)

Review 0 of 15833


C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Review 1000 of 15833
Review 2000 of 15833
Review 3000 of 15833
Review 4000 of 15833
Review 5000 of 15833
Review 6000 of 15833
Review 7000 of 15833
Review 8000 of 15833
Review 9000 of 15833
Review 10000 of 15833
Review 11000 of 15833
Review 12000 of 15833
Review 13000 of 15833
Review 14000 of 15833
Review 15000 of 15833


In [15]:
# Calculating average feature vector for training set
clean_train_reviews1 = []
for review in train1['content']:
    clean_train_reviews1.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs1 = getAvgFeatureVecs(clean_train_reviews1, model1, num_features1)
#print (trainDataVecs)

Review 0 of 4026


C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Review 1000 of 4026
Review 2000 of 4026
Review 3000 of 4026
Review 4000 of 4026


In [16]:
# Calculating average feature vactors for test set     
clean_test_reviews = []
for review in test["Tweet"]:
    clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 4068


C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\Talha Saleem\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


Review 1000 of 4068
Review 2000 of 4068
Review 3000 of 4068
Review 4000 of 4068


In [17]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
print (tokenizer)
tokenizer.fit_on_texts(train['content'].values)
X = tokenizer.texts_to_sequences(train["content"].values)
X = pad_sequences(X)
print (X.shape[1])

28


In [18]:
max_features = 2000
tokenizer1 = Tokenizer(num_words=max_features, split=' ')
print (tokenizer1)
tokenizer1.fit_on_texts(train1['content'].values)
X1 = tokenizer1.texts_to_sequences(train1["content"].values)
X1 = pad_sequences(X1)
print (X1.shape[1])

27


In [19]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

W0310 16:42:57.600806  2480 deprecation_wrapper.py:119] From C:\Users\Talha Saleem\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0310 16:42:59.189522  2480 deprecation_wrapper.py:119] From C:\Users\Talha Saleem\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0310 16:43:00.051930  2480 deprecation_wrapper.py:119] From C:\Users\Talha Saleem\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0310 16:43:00.964685  2480 deprecation_wrapper.py:119] From C:\Users\Talha Saleem\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0310 16:43:01.046

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
embed_dim1 = 128
lstm_out1 = 196

model1 = Sequential()
model1.add(Embedding(max_features, embed_dim1,input_length = X1.shape[1]))
model1.add(SpatialDropout1D(0.4))
model1.add(LSTM(lstm_out1, dropout=0.2, recurrent_dropout=0.2))
model1.add(Dense(4,activation='softmax'))
model1.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 27, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 788       
Total params: 511,588
Trainable params: 511,588
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
Y = pd.get_dummies(train['Affect']).values #thissssssssssssssssssssssssssssssssssssssssssssssssss
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)

In [22]:
Y1 = pd.get_dummies(train1['Intensity']).values #thissssssssssssssssssssssssssssssssssssssssssssssssss
X1_train, X1_test, Y1_train, Y1_test = train_test_split(X1,Y1, test_size = 0.3, random_state = 42)


In [23]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 4, batch_size=batch_size, verbose = 2)

W0310 16:43:16.296323  2480 deprecation.py:323] From C:\Users\Talha Saleem\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/4
 - 142s - loss: 0.9977 - acc: 0.5647
Epoch 2/4
 - 162s - loss: 0.6742 - acc: 0.7389
Epoch 3/4
 - 105s - loss: 0.5711 - acc: 0.7781
Epoch 4/4
 - 216s - loss: 0.5203 - acc: 0.8024


In [24]:
batch_size = 32
model1.fit(X1_train, Y1_train, epochs = 3, batch_size=batch_size, verbose = 2)

Epoch 1/3
 - 27s - loss: 1.1491 - acc: 0.3378
Epoch 2/3
 - 18s - loss: 1.0485 - acc: 0.4319
Epoch 3/3
 - 20s - loss: 0.8873 - acc: 0.5994


In [25]:
"""# save the model to disk
filename_emotion = 'emotion_model.sav'
pickle.dump(model, open(filename_emotion, 'wb'))
filename_intensity = 'intensity_model.sav'
pickle.dump(model, open(filename_intensity, 'wb'))"""

"# save the model to disk\nfilename_emotion = 'emotion_model.sav'\npickle.dump(model, open(filename_emotion, 'wb'))\nfilename_intensity = 'intensity_model.sav'\npickle.dump(model, open(filename_intensity, 'wb'))"

In [26]:
"""# load the model from disk
loaded_model = pickle.load(open(filename_emotion, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)"""

"# load the model from disk\nloaded_model = pickle.load(open(filename_emotion, 'rb'))\nresult = loaded_model.score(X_test, Y_test)\nprint(result)"

In [27]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
print (X_test.shape)
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

(3250, 28)
score: 0.68
acc: 0.73


In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [29]:
twt = ["i am insulted"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)



print ("twt:", twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
twt1 = pad_sequences(twt, maxlen=27, dtype='int32', value=0)
    
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
Intensity = model1.predict(twt1,batch_size=1,verbose = 2)[0]
    
    
label=int(np.argmax(sentiment))
intensity=int(np.argmax(Intensity))



print (label)
print (intensity)

twt: [[2, 53]]
2
2


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [ ]:
from _thread import *
from socket import *
import threading 
import socket
import json
from io import StringIO
twt_add=[]
def threaded(c):
    data = c.recv(1024)
    data = json.loads( data.decode("utf-8")  )
    print(data)

    twt_add.append(str(data["Sentence"]))
    #print("twt: ",str(twt_add))
    #vectorizing the tweet by the pre-fitted tokenizer instance
    twt = tokenizer.texts_to_sequences(twt_add)
    del twt_add[-1]
    print ("After tokenizing: ", twt_add)
    
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
    twt1 = pad_sequences(twt, maxlen=27, dtype='int32', value=0)

    sentiment = model.predict(twt,batch_size=1,verbose = 2)
    Intensity = model1.predict(twt1,batch_size=1,verbose = 2)


    
    
    #print("Labelsssss: ", sentiment )
    #print ("Intensityssss",intensity)
    
    
    label1=int(np.argmax(sentiment))
    intensity1=int(np.argmax(Intensity))

    
    #print("Label1: ", label1 )
    #print ("Intensity1",intensity1)
    intensity1+=1
    
    #vector = count_vector.transform( [data["Sentence"]] )
    label_1 = str(label1)
    intensity_1=str(intensity1)
    #print("Label_1: ", label_1 )
    #print ("Intensity_1",intensity_1)
    concat= label_1+","+intensity_1
    c.send(  json.dumps({"label": (concat) }).encode('utf-8')   )
    c.close()
port = 5020
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
s.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
s.bind((  "0.0.0.0" , port  )) 
print("socket binded to port", port )
s.listen(5) 
print("socket is listening")
all_conn = []
while True:
    conn, addr = s.accept()
    print('Connected to :', addr[0], ':', addr[1])
    all_conn.append( conn )
    start_new_thread(threaded, (conn,))
s.close()

socket binded to port 5020
socket is listening
Connected to : 192.168.100.18 : 48308
{'Sentence': 'Im very sad today'}
After tokenizing:  []
